### LCEL中 Memory 的添加方式

In [2]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_community.chat_models.tongyi import ChatTongyi
from operator import itemgetter

model = ChatTongyi(model="qwen-plus")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是一个乐于助人的机器人"),
        MessagesPlaceholder(variable_name="history"),
        ("human","{input}")
    ]
)

memory = ConversationBufferMemory(return_messages=True)

memory.load_memory_variables({})

{'history': []}

In [8]:
# 增加一条链

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(lambda _: memory.load_memory_variables({})["history"])
    )
    | RunnablePassthrough.assign(
        input=itemgetter("input")
    )
    | prompt
    | model
)

In [9]:
inputs = {"input":"你好，我是Mach"}
response = chain.invoke(inputs)
response

AIMessage(content='你好，Mach！很高兴认识你。如果你有任何问题或需要帮助的地方，请随时告诉我！', additional_kwargs={}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': 'a976b048-d581-92b8-9933-b95f978c7c58', 'token_usage': {'input_tokens': 25, 'output_tokens': 20, 'total_tokens': 45, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--ea089813-4607-4b0b-801b-f1cf7d05cba9-0')

In [10]:
# 保存记忆
memory.save_context(inputs,{"output":response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='你好，我是Mach', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好，Mach！很高兴认识你。如果你有任何问题或需要帮助的地方，请随时告诉我！', additional_kwargs={}, response_metadata={})]}

In [11]:
inputs = {"input":"你好，我是谁？"}
response = chain.invoke(inputs)
response

AIMessage(content='你好！从我们的对话来看，你目前使用的是名字“Mach”。不过，如果你在思考更深层次的“我是谁”这个问题，那可能涉及到哲学、身份认同或者个人兴趣等方面。你可以告诉我更多背景信息，这样我就能更好地回答你的问题啦！\n\n如果是开玩笑的话——哈哈，你就是那个正在和我聊天的有趣的人呀！ 😊', additional_kwargs={}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'stop', 'request_id': 'e1bedc5e-0c6c-9a36-a20a-e32bbf752f15', 'token_usage': {'input_tokens': 60, 'output_tokens': 77, 'total_tokens': 137, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--fe6ebf6d-4113-46fd-a80b-5e51ca224840-0')

#### 使用Redis实现长时记忆

In [12]:
! pip install redis

In [13]:
from typing import Optional
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是一个擅长{ability}的助手"),
        MessagesPlaceholder(variable_name="history"),
        ("human","question")
    ]
)

model = ChatTongyi(model="qwen-plus",temperature=0)

chain = prompt | model

In [15]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    # 使用redis存储聊天记录
    lambda session_id: RedisChatMessageHistory(session_id=session_id,url="..."),
    input_messages_key="question",
    history_messages_key="history"
)

In [ ]:
# 每次调用都会保存聊天记录，需要有对应的session_id

chain_with_history.invoke(
    {
        "ability":"历史",
        "question":"中国建都时间最久的城市是哪个？"
    },
    config=
    {
        "configurable":
        {
            "session_id":"..."
        }
    }
)

In [ ]:
chain_with_history.invoke(
    {
        "ability":"历史",
        "question":"它有多少年建都历史？"
    },
    config=
    {
        "configurable":
        {
            "session_id":"..."
        }
    }
)